### Quantum Anomaly Detection for Network Security

We’ll build a binary classifier that detects whether network traffic is normal or anomalous (malicious) using:
- ✅ Classical model: Support Vector Machine (SVM)
- ⚛️ Quantum models:
- QSVC (Quantum Kernel SVM)
- VQC (Variational Quantum Classifier)

All models will be trained on a real-world cybersecurity dataset: [CICIDS2017](https://www.unb.ca/cic/datasets/ids-2017.html).

In [8]:
import pandas as pd

# Load dataset
df = pd.read_csv("CICIDS2017_sample.csv")

# Drop missing values
df = df.dropna()

# Label: convert to binary (0 = Normal, 1 = Attack)
df["Label"] = df["Label"].apply(lambda x: 0 if x == "BENIGN" else 1)

# Separate features and labels
X = df.drop("Label", axis=1).values
y = df["Label"].values

In [9]:
df.count()

Flow Duration             100
Total Fwd Packets         100
Total Backward Packets    100
Flow Bytes/s              100
Flow Packets/s            100
Label                     100
dtype: int64